Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.

The input features will be radius and height and the label will be $\pi r^2 h$

In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil
import math

In [14]:
# create datasets: train, validation, test
df_train = pd.DataFrame(1.5 * np.random.random_sample((10000,2)) + 0.5, 
            columns = ['radius','height'])

df_valid = pd.DataFrame(1.5 * np.random.random_sample((1000,2)) + 0.5, 
            columns = ['radius','height'])

df_test = pd.DataFrame(1.5 * np.random.random_sample((1000,2)) + 0.5, 
            columns = ['radius','height'])

In [15]:
# Calculate and append volume to datasets

# training data
labels = []
for index, row in df_train.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_train.insert(0, 'volume', labels, True)

# validation data
labels.clear()
for index, row in df_valid.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_valid.insert(0, 'volume', labels, True)

# test data
labels.clear()
for index, row in df_test.iterrows():
  volume = math.pi * row['radius']**2 * row['height']
  labels.append(volume)
df_test.insert(0, 'volume', labels, True)

In [16]:
# make input function

def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [17]:
# make prediction function

def make_prediction_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = False,            # shuffle = False so we can compare predictions with actual
    queue_capacity = 1000,
    num_threads = 1
  )

In [18]:
# make feature columns function

DF_COLUMNS = ['volume','radius', 'height']
FEATURES = DF_COLUMNS[1:len(DF_COLUMNS)]
LABEL = DF_COLUMNS[0]

def make_feature_cols():
    input_cols = [tf.feature_column.numeric_column(i) for i in FEATURES]
    return input_cols

In [19]:
# method for determining RMSE

def print_rmse (model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))

In [20]:
# set directory for trained model, instantiate model, and train model

OUTDIR = 'cylinder_trained'


# for debugging purposes
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True)


# instantiate model
model = tf.estimator.DNNRegressor(
  hidden_units = [32, 8, 2],
  feature_columns = make_feature_cols(),
  model_dir = OUTDIR)


# train model
model.train(input_fn = make_input_fn(df_train, num_epochs = 100))

print_rmse(model, 'validation', df_valid)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cylinder_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x117505f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into cylinder_trained/model.ckpt.
INFO:tensorflow:loss = 9057.927

INFO:tensorflow:global_step/sec: 622.282
INFO:tensorflow:loss = 1.6771419, step = 7401 (0.161 sec)
INFO:tensorflow:global_step/sec: 626.084
INFO:tensorflow:loss = 1.2469071, step = 7501 (0.159 sec)
INFO:tensorflow:global_step/sec: 632.604
INFO:tensorflow:loss = 1.4097581, step = 7601 (0.159 sec)
INFO:tensorflow:global_step/sec: 633.982
INFO:tensorflow:loss = 1.5113006, step = 7701 (0.158 sec)
INFO:tensorflow:global_step/sec: 632.091
INFO:tensorflow:loss = 1.9940252, step = 7801 (0.158 sec)
INFO:tensorflow:Saving checkpoints for 7813 into cylinder_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 1.0455253.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-29-19:46:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cylinder_trained/model.ckpt-7813
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-29-19

In [21]:
# create predictions generator

predictions = model.predict(input_fn = make_prediction_input_fn(df_valid, 1))

In [22]:
# print first 20 entries from the validation set with their actual volume and predicted volume

for index, row in df_valid.iterrows():
    r = row['radius']
    h = row['height']
    v = math.pi * r**2 * h
    raw = next(predictions)
    p = raw['predictions'][0]
    d = abs(((p-v)/v) * 100)
    
    print("\nCylinder ",index+1)
    print("radius     = {0:.3f}\nheight     = {1:.3f}\nvolume "
                "    = {2:.3f}\nprediction = {3:.3f}\ndifference = {4:.3f}%".format(r,h,v,p,d))
    if index == 19:
        break
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cylinder_trained/model.ckpt-7813
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Cylinder  1
radius     = 1.270
height     = 0.926
volume     = 4.693
prediction = 4.694
difference = 0.028%

Cylinder  2
radius     = 1.688
height     = 1.955
volume     = 17.498
prediction = 17.581
difference = 0.474%

Cylinder  3
radius     = 0.937
height     = 1.337
volume     = 3.689
prediction = 3.796
difference = 2.880%

Cylinder  4
radius     = 0.576
height     = 1.806
volume     = 1.881
prediction = 1.981
difference = 5.322%

Cylinder  5
radius     = 1.286
height     = 0.807
volume     = 4.195
prediction = 4.289
difference = 2.236%

Cylinder  6
radius     = 0.537
height     = 1.658
volume     = 1.504
prediction = 1.580
difference = 5.072%

Cylinder  7
radius     = 1.957
height     = 1.988
volume     = 23.928
predi